# Sistema de Recomendação de Cartões de Crédito

Este notebook tem como objetivo:

1. Desenvolver um modelo preditivo para classificação do principal cartão para clientes
2. Aplicar o modelo de clientes na base prospects
3. Gerar arquivo final com cartão ideal (recomendação) para prospects

### Importar bibliotecas necessárias

In [1]:
# EDA e Visualização de Dados
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency, f_oneway
from colorama import Fore, Back, Style

# Configurar formato de exibição para não usar notação científica
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True, precision=5)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# ML
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, log_loss
from catboost import CatBoostClassifier, Pool, cv

# Otimização
import optuna

# Utilitários
import joblib

/home/emerson/.local/share/virtualenvs/final_challenge-Iu6pVGne/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Abrir Base Clientes

In [2]:
df_clientes = pd.read_csv('./dataset/clientes.csv')

In [3]:
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID_Cliente                   1000 non-null   object 
 1   Nome                         1000 non-null   object 
 2   Idade                        1000 non-null   int64  
 3   Viagens                      1000 non-null   int64  
 4   Restaurantes                 1000 non-null   int64  
 5   Entretenimento               1000 non-null   int64  
 6   Cashback                     1000 non-null   int64  
 7   Compras online               1000 non-null   int64  
 8   Farmácias                    1000 non-null   int64  
 9   Programas de Milhagem        1000 non-null   int64  
 10  Postos de Combustível        1000 non-null   int64  
 11  Mercados                     1000 non-null   int64  
 12  Cidade                       1000 non-null   object 
 13  Cargo              

In [4]:
# Remover colunas únicas
df_clientes.drop(columns=['ID_Cliente', 'Nome'], axis=1, inplace=True)

In [5]:
num_vars = df_clientes.select_dtypes(include=['number']).columns
cat_vars = df_clientes.select_dtypes(exclude=['object']).columns
target = 'Principal Cartão'

### Testes de Hipóteses

In [6]:
# Testes de hipóteses entre Target Categórica e Numéricas (ANOVA)
for num_col in num_vars:
  groups = [df_clientes[df_clientes[target] == val][num_col] for val in df_clientes[target].unique()]
  stat, p = f_oneway(*groups)
  print(f"{Fore.RED if p < 0.05 else Fore.WHITE}"
        f"ANOVA entre {num_col} e {target}: p-valor = {p}")

ANOVA entre Idade e Principal Cartão: p-valor = 0.02334622684107804
ANOVA entre Viagens e Principal Cartão: p-valor = 4.59308651849609e-78
ANOVA entre Restaurantes e Principal Cartão: p-valor = 6.679790462858737e-13
ANOVA entre Entretenimento e Principal Cartão: p-valor = 5.089212006248722e-18
ANOVA entre Cashback e Principal Cartão: p-valor = 8.77437441029354e-12
ANOVA entre Compras online e Principal Cartão: p-valor = 0.00717148660617559
ANOVA entre Farmácias e Principal Cartão: p-valor = 0.0006390735697304401
ANOVA entre Programas de Milhagem e Principal Cartão: p-valor = 0.001269305962592174
ANOVA entre Postos de Combustível e Principal Cartão: p-valor = 0.11255638879254316
ANOVA entre Mercados e Principal Cartão: p-valor = 0.034152908342727085
ANOVA entre Tempo na Empresa e Principal Cartão: p-valor = 0.3209237036500026
ANOVA entre Pessoas em Casa e Principal Cartão: p-valor = 0.05946641455988785
ANOVA entre Renda e Principal Cartão: p-valor = 1.5360217142979344e-34
ANOVA entre In

In [7]:
# Analisando relação entre variáveis explicativas e targets
for col in num_vars:
  fig = px.box(df_clientes, x=target, y=col, title=f"{col} por {target}")
  fig.show()

for col in cat_vars:
  fig = px.histogram(df_clientes, x=col, color=target, title=f"{col} por {target}")
  fig.show()